In [ ]:
# Three datasets are created 
# These Datasets are created after cleaning all the datasets provided 
# All_PATIENTS.csv This Dataset contains all 25 patients data from HCPA000*P.csv
# DEMOGRAPHIC.csv This dataset contains demogrpahic data provided in  "T1DM_patient_sleep_demographics_with_race.csv"
# PATINETS_WITH_DEMOGRAPHIC.csv This dataset is merge of above two datasets
# This Jupyter notebooks has cells:
    # importing modules
    # declaring variables
    # creating a dataframe by merging all csv files having patient data
    # change time into date and time and write the clean data into csv file
    # dataframe for demographic data by renaming all column names to lowercase with underscore betweeen words and created csv file
    # dataframe for merged allpatients and demographic data and created csv file

In [3]:
#importing all the necessary libraries
import pandas as pd
import glob
import os
import numpy as np
import warnings
warnings.simplefilter("ignore", UserWarning)

In [109]:
#global variables
folder_path="patients_data"
file_pattern="HUPA*.csv",
delimiter=";"
#Variables
#path = "../HUPA-UC Diabetes Dataset/"
demographic_path = folder_path + "T1DM_patient_sleep_demographics_with_race.csv"
#all_patients_path = path + "ALL_PATIENTS.csv"
#modified_demographic_path = path + "DEMOGRAPHIC.csv"
#patients_demographic_path = path +"PATIENTS_WITH_DEMOGRAPHIC.csv"

combined all 25 patient files. cleaned and saved to the final_patients.csv file
Removes any repeated rows.
Why: Duplicate entries can skew statistics, especially in time-series or patient-level analysis.
Drops rows where either time or glucose is missing.
Why: These are critical fields for tracking blood sugar trends. Without them, the data point is unusable.
Converts the time column to the proper datetime format.
Why: Ensures consistency for sorting, filtering, and time-based feature engineering.
Note: errors='coerce' turns invalid formats into NaT (missing), which is handled in the next step
Removes rows where time couldn't be parsed.
Why: Keeps only rows with valid timestamps for chronological analysis.
Sorts the data by patient and time, then resets the index.
Prepares the dataset for time-series modeling, rolling averages, or lag feature creation per patient.
No duplicate or incomplete records
Valid timestamps for temporal analysis
Chronological ordering per patient

 Step 1: Load Existing Final File (if it exists)

In [65]:
import pandas as pd
import os

# Load existing final file
final_path = "final_patients.csv"
if os.path.exists(final_path):
    existing_df = pd.read_csv(final_path)
    print(f"✅ Loaded existing file with {len(existing_df)} records")
else:
    existing_df = pd.DataFrame()
    print("📂 No existing file found. Starting fresh.")


📂 No existing file found. Starting fresh.


Step 2: Load New Patient Files and combine all the 25 files


In [71]:
import glob

# Load new files
folder_path = "patients_data"
file_pattern = "HUPA*.csv"
file_list = glob.glob(os.path.join(folder_path, file_pattern))
print(f"📁 Found {len(file_list)} patient files")

# Load and tag each file
df_list = []
for file in file_list:
    temp_df = pd.read_csv(file, delimiter=";")
    temp_df['patient_id'] = os.path.basename(file).split(".")[0]
    df_list.append(temp_df)

# Combine all new files
new_df = pd.concat(df_list, ignore_index=True)
print(f"🆕 Loaded new batch with {len(new_df)} records")


📁 Found 25 patient files
🆕 Loaded new batch with 309392 records


 Step 3: Clean New Data
Removes any repeated rows.
Why: Duplicate entries can skew statistics, especially in time-series or patient-level analysis.
Drops rows where either time or glucose is missing.
Why: These are critical fields for tracking blood sugar trends. Without them, the data point is unusable.
Converts the time column to the proper datetime format.
Why: Ensures consistency for sorting, filtering, and time-based feature engineering.
Note: errors='coerce' turns invalid formats into NaT (missing), which is handled in the next step
Removes rows where time couldn't be parsed.
Why: Keeps only rows with valid timestamps for chronological analysis.
Sorts the data by patient and time, then resets the index.
Prepares the dataset for time-series modeling, including rolling averages and lag feature creation, per patient.
No duplicate or incomplete records
Valid timestamps for temporal analysis
Chronological ordering per patient

In [73]:
new_df = new_df.drop_duplicates()
new_df = new_df.dropna(subset=['time', 'glucose'])
new_df['time'] = pd.to_datetime(new_df['time'], errors='coerce')
new_df = new_df.dropna(subset=['time'])
new_df = new_df.sort_values(by=['patient_id', 'time']).reset_index(drop=True)
print(f"🧼 Cleaned new data: {len(new_df)} records remain")


🧼 Cleaned new data: 309392 records remain


Step 4: Merge and Deduplicate
Here’s the critical part — we’ll deduplicate based on key columns only:
This ensures that even if the same file is reprocessed, it won’t duplicate rows unless glucose or time differs

In [93]:
# Merge and deduplicate
combined_df = pd.concat([existing_df, new_df], ignore_index=True)
combined_df = combined_df.drop_duplicates(subset=['patient_id', 'time', 'glucose']).reset_index(drop=True)
print(f"🔗 Combined total: {len(combined_df)} records after deduplication")


🔗 Combined total: 309392 records after deduplication


Splitting time into date and time helps correlate patient events with demographic factors like 
age, gender, or region by enabling daily or hourly trend analysis. 
It also supports clinical insights—such as identifying post-meal glucose spikes, sleep-related vitals, 
or medication timing effects—by aligning temporal patterns with medical context.


In [95]:
# Split 'time' into 'date' and 'time_only'
combined_df['date'] = combined_df['time'].dt.date
combined_df['time_only'] = combined_df['time'].dt.time

# Reorder columns: patient_id, date, time, then the rest
ordered_cols = ['patient_id', 'date', 'time_only'] + [
    col for col in combined_df.columns 
    if col not in ['patient_id', 'date', 'time_only', 'time']
]
combined_df = combined_df[ordered_cols]

# Rename 'time_only' back to 'time' for clarity
combined_df.rename(columns={'time_only': 'time'}, inplace=True)

print("✅ Final reshaped data saved to 'final_patients.csv'")


✅ Final reshaped data saved to 'final_patients.csv'


Step-by-Step Precision Cleaning Code
This step applies controlled rounding to selected numeric columns to improve readability, reduce noise from excessive precision, 
and align with clinical reporting standards. It excludes columns that already meet the desired format to avoid redundancy. 
The goal is to create a clean, interpretable dataset that supports reliable analysis and downstream modeling.

In [197]:
# Define only the columns you want to clean
columns_to_clean = [
    'calories', 'heart_rate', 'basal_rate'
]

# Precision map for those columns
precision_map = {
    'calories': 2,
    'heart_rate': 1,
    'basal_rate': 4,
}

# Drop all existing _cleaned columns to start fresh
combined_df.drop(columns=[col for col in combined_df.columns if col.endswith('_cleaned')], inplace=True)

# Apply rounding only to selected columns
for col in columns_to_clean:
    if col in combined_df.columns and pd.api.types.is_numeric_dtype(combined_df[col]):
        combined_df[f"{col}_cleaned"] = combined_df[col].round(precision_map[col])

print(combined_df.head().to_string(index=False))
print(combined_df.columns.tolist())


patient_id       date     time  glucose  calories  heart_rate  steps  basal_rate  bolus_volume_delivered  carb_input  calories_cleaned  heart_rate_cleaned  basal_rate_cleaned
 HUPA0001P 2018-06-13 18:40:00    332.0    6.3595   82.322835   34.0    0.091667                     0.0         0.0              6.36                82.3              0.0917
 HUPA0001P 2018-06-13 18:45:00    326.0    7.7280   83.740157    0.0    0.091667                     0.0         0.0              7.73                83.7              0.0917
 HUPA0001P 2018-06-13 18:50:00    330.0    4.7495   80.525180    0.0    0.091667                     0.0         0.0              4.75                80.5              0.0917
 HUPA0001P 2018-06-13 18:55:00    324.0    6.3595   89.129032   20.0    0.091667                     0.0         0.0              6.36                89.1              0.0917
 HUPA0001P 2018-06-13 19:00:00    306.0    5.1520   92.495652    0.0    0.075000                     0.0         0.0         

 Step 5: Save Final File

In [188]:
combined_df.to_csv(final_path, index=False)
print(f"✅ Final saved to {final_path}")


✅ Final saved to final_patients.csv


 Step 6: Preview the Final CSV

In [190]:
# Load and preview the saved final file
final_df = pd.read_csv(final_path)

# Basic info
print(f"📊 Final file has {len(final_df)} records and {final_df.shape[1]} columns")

# Show first few rows
print(final_df.head().to_string(index=False))


📊 Final file has 309392 records and 13 columns
patient_id       date     time  glucose  calories  heart_rate  steps  basal_rate  bolus_volume_delivered  carb_input  calories_cleaned  heart_rate_cleaned  basal_rate_cleaned
 HUPA0001P 2018-06-13 18:40:00    332.0    6.3595   82.322835   34.0    0.091667                     0.0         0.0              6.36                82.3              0.0917
 HUPA0001P 2018-06-13 18:45:00    326.0    7.7280   83.740157    0.0    0.091667                     0.0         0.0              7.73                83.7              0.0917
 HUPA0001P 2018-06-13 18:50:00    330.0    4.7495   80.525180    0.0    0.091667                     0.0         0.0              4.75                80.5              0.0917
 HUPA0001P 2018-06-13 18:55:00    324.0    6.3595   89.129032   20.0    0.091667                     0.0         0.0              6.36                89.1              0.0917
 HUPA0001P 2018-06-13 19:00:00    306.0    5.1520   92.495652    0.0    0.0750

rename the T1DM_patient_sleep_demographics_with_race file column names

In [192]:
# Load demographics data
df_demo = pd.read_csv(os.path.join("patients_data", "T1DM_patient_sleep_demographics_with_race.csv"))
# renaming column nmaes
rename_map = {
    'Patient_ID': 'patient_id',
    'Age': 'age',
    'Gender': 'gender',
    'Race': 'race',
    'Average Sleep Duration (hrs)': 'avg_sleep_duration_hrs',
    'Sleep Quality (1-10)': 'sleep_quality_score',
    '% with Sleep Disturbances': 'sleep_disturbances_percentage'
}

df_demo.rename(columns=rename_map, inplace=True)
#saving the data to the cleaned_demographics.csv file 
df_demo.to_csv("cleaned_demographics.csv", index=False)
print(df_demo.head().to_string(index=False))

patient_id  age gender            race  avg_sleep_duration_hrs  sleep_quality_score  sleep_disturbances_percentage
 HUPA0001P   34   Male           Other                     6.3                  4.5                             80
 HUPA0002P   49   Male        Hispanic                     6.6                  4.4                             40
 HUPA0003P   64   Male           Black                     5.3                  5.2                             70
 HUPA0004P   34 Female Native American                     5.2                  6.9                             60
 HUPA0005P   49   Male Native American                     5.8                  7.9                             30


combining cleaned_demographies and all_patients data

In [194]:
import pandas as pd
import os

# Step 1: Load final patients data
df_patients = pd.read_csv(final_path)

# Step 2: Load demographics data
df_demograph = pd.read_csv("cleaned_demographics.csv")

# Step 3: Merge on patient_id
df_merged = pd.merge(df_patients, df_demograph, on="patient_id", how="left")

# Step 4: Preview merged data
print("🔗 Merged Data Summary")
print(f"Total records: {len(df_merged)}")
print(f"Columns: {df_merged.columns.tolist()}")
print("🧪 Sample rows:")
print(df_merged.head().to_string(index=False))

# Step 5: Save merged version
df_merged.to_csv("patients_demography.csv", index=False)


🔗 Merged Data Summary
Total records: 309392
Columns: ['patient_id', 'date', 'time', 'glucose', 'calories', 'heart_rate', 'steps', 'basal_rate', 'bolus_volume_delivered', 'carb_input', 'calories_cleaned', 'heart_rate_cleaned', 'basal_rate_cleaned', 'age', 'gender', 'race', 'avg_sleep_duration_hrs', 'sleep_quality_score', 'sleep_disturbances_percentage']
🧪 Sample rows:
patient_id       date     time  glucose  calories  heart_rate  steps  basal_rate  bolus_volume_delivered  carb_input  calories_cleaned  heart_rate_cleaned  basal_rate_cleaned  age gender  race  avg_sleep_duration_hrs  sleep_quality_score  sleep_disturbances_percentage
 HUPA0001P 2018-06-13 18:40:00    332.0    6.3595   82.322835   34.0    0.091667                     0.0         0.0              6.36                82.3              0.0917   34   Male Other                     6.3                  4.5                             80
 HUPA0001P 2018-06-13 18:45:00    326.0    7.7280   83.740157    0.0    0.091667            

In [ ]:
# Step 3: Apply rounding only if current precision exceeds recommended and creates a new column for each one required precission
